To generate template for new proposals.

In [1]:
#%%
import os
import re
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import xlwings as xw

import functions
import hide

In [2]:
# Initialize the require parameters.
# file name does not require file extension. File will be saved to Downloads folder.
# system_names cannot exceed the character length of 32 as this is excel's sheet names limit.
# system_names should be put in required order.
file_name = 'J12234 MODEC- EQUINOR BACALHAU FPSO - VSAT MPOWER V6-R0'
system_names = ['VSAT MPOWER SYSTEM', 'START-UP & COMMISSIONING SPARES', 'TWO YEARS SPARES', 'ENGINEERING SERVICES']
project_info = {'Attend to: ' : "ANDREW MAK",
                'Designation: ' : "PROCUREMENT",
                'Client Name: ' : "MODEC",
                'Client RFQ No: ' : "AS PER EMAIL REQUEST",
                'Ref Doc No: ' : "AS PER EMAIL",
                'Project Name: ' : "EQUINOR BACALHAU FPSO - VSAT MPOWER",
                'Prepared By: ' : "THIHA AUNG",
                'Sales Manager: ' : "SHAUN TEO",
                'Jason Ref: ' : 'J12234 V6',
                'Revision Num: ' : 'R0',
                'Comn Site: ' : 'SINGAPORE'}
quoted_currency = 'USD'
give_discount = True

In [3]:
downloads_folder = os.path.join(os.path.expanduser('~'), 'Downloads')
file_path = Path(downloads_folder, file_name)
# Download template file from the internet and write to local folder
url = "https://filedn.com/liTeg81ShEXugARC7cg981h/Proposal_Template.xlsx"
resp = requests.get(url)

with open("Template.xlsx", 'wb') as fd:
    for chunk in resp.iter_content(chunk_size=8192):
        fd.write(chunk)
system_names = [x.upper() for x in system_names]
# Copy sheet from template to new workbook
new_book = xw.Book()
template = xw.Book("Template.xlsx", password=hide.new)
template.sheets['config'].copy(after=new_book.sheets[0])
new_book.sheets['Sheet1'].delete()
template.sheets['Cover'].copy(after=new_book.sheets['config'])

# Project info
# Set exchange rates
sheet = new_book.sheets['Config']
# Quoted currency
sheet.range('B12').value = quoted_currency
# Project info
sheet.range('B21').value = project_info['Attend to: ']
sheet.range('B22').value = project_info['Designation: ']
sheet.range('B23').value = project_info['Client Name: ']
sheet.range('B24').value = project_info['Client RFQ No: ']
sheet.range('B25').value = project_info['Ref Doc No: ']
sheet.range('B26').value = project_info['Project Name: ']
sheet.range('B27').value = project_info['Prepared By: ']
sheet.range('B28').value = project_info['Sales Manager: ']
sheet.range('B29').value = project_info['Jason Ref: ']
sheet.range('B30').value = project_info['Revision Num: ']
# sheet.range('B31').value = project_info['']
sheet.range('B32').value = datetime.today().strftime('%Y-%m-%d')

# Set up formula in Cover sheet
sheet = new_book.sheets['Cover']
sheet.range('D7').formula = '=Config!B26'
sheet.range('C42').formula = '=Config!B21'
sheet.range('C43').formula = '=Config!B23'
sheet.range('C44').formula = '=Config!B24'
sheet.range('C45').formula = '=Config!B29'
sheet.range('C46').formula = '=Config!B30'
sheet.range('C47').formula = '=Config!B32'
sheet.range('D39').formula = '=Config!B13'

for system in system_names[::-1]:
    sheet_name = 'Cover'
    template.sheets['System'].copy(after=new_book.sheets[sheet_name])
    sheet_name = system
    new_book.sheets['System'].name = sheet_name
    # Set formula to reference Config.
    new_book.sheets[sheet].range('A1').formula = '= "JASON REF: " & Config!B29 &  ", REVISION: " &  Config!B30 & ", PROJECT: " & Config!B26'
    # new_book.sheets[sheet_name].range('C1').formula = '=Config!B29'
    # new_book.sheets[sheet_name].range('C2').formula = '=Config!B30'
    # new_book.sheets[sheet_name].range('C3').formula = '=Config!B32'
    # new_book.sheets[sheet_name].range('C4').formula = '=Config!B26'
template.sheets['Summary'].copy(after=new_book.sheets['Cover'])
template.sheets['Technical_Notes'].copy(after=new_book.sheets[-1])
template.sheets['T&C'].copy(after=new_book.sheets[-1])
for sheet in new_book.sheet_names:
    if sheet in ['Summary','Technical_Notes', 'T&C']:
        new_book.sheets[sheet].range('A1').formula = '= "JASON REF: " & Config!B29 &  ", REVISION: " &  Config!B30 & ", PROJECT: " & Config!B26'
        # new_book.sheets[sheet].range('C1').formula = '=Config!B29'
        # new_book.sheets[sheet].range('C2').formula = '=Config!B30'
        # new_book.sheets[sheet].range('C3').formula = '=Config!B32'
        # new_book.sheets[sheet].range('C4').formula = '=Config!B26'
template.close()

In [4]:
# Write necessary formula to excel
for system in system_names:
    sheet = new_book.sheets[system]

    # Set default values
    if system == 'ENGINEERING SERVICES':
        sheet.range('J1').value = 0.3
        sheet.range('L1').value = 0
        sheet.range('N1').value = 0
        sheet.range('P1').value = 0
        sheet.range('R1').value = 0
    
    functions.fill_formula(new_book, sheet)
    functions.format(new_book, sheet)

In [5]:
summary_formula = []
collect = [] # Collect formula to be put in summary page.
formula_fragment = '=IF(OR(Config!B13="COMMERCIAL PROPOSAL", Config!B13="BUDGETARY PROPOSAL"),'
for sheet in system_names:
    sheet = new_book.sheets[sheet]
    last_row = sheet.range('G100000').end('up').row
    collect = [ formula_fragment + "'" + sheet.name + "'!$G$" + str(last_row) + ', "")',
               formula_fragment + "'" + sheet.name + "'!$U$" + str(last_row) + ', "")']
            #    "='" + sheet.name + "'!$AF$" + str(last_row)]
    summary_formula.extend(collect)
    collect = []

count = 1
offset = 20
odered_summary_formula = summary_formula[::-1]
sheet = new_book.sheets['Summary']
for system in system_names:
    sheet.range('B' + str(offset)).value = count
    sheet.range('C' + str(offset)).value = system
    sheet.range('D' + str(offset)).formula = odered_summary_formula.pop()
    # if new_book.sheets['Config'].range("B13").value in ["COMMERCIAL PROPOSAL", "BUDGETARY PROPOSAL"]
    sheet.range('H' + str(offset)).formula = odered_summary_formula.pop()
    sheet.range('I' + str(offset)).formula = '=IF(H'+ str(offset) + '<>"",D' + str(offset) + '- H' + str(offset) + ',"")'
    sheet.range('J' + str(offset)).formula = '=IF(I' + str(offset) + '<>"",I' + str(offset) + '/D' + str(offset) + ',"")'
    count += 1
    offset += 1

(new_book.sheets['Config'].range('106:106')).copy(sheet.range(str(offset) + ':' + str(offset)))
(new_book.sheets['Config'].range('102:102')).copy(sheet.range(str(offset+1) + ':' + str(offset+1)))
sheet = new_book.sheets['Summary']
sheet.range('C' + str(offset+1)).value = '="TOTAL PROJECT (" & Config!B12 & ")"'
sheet.range('D' + str(offset+1)).formula = '=SUMIF(E20:E' + str(offset) + ',"<>OPTION",D20:D' + str(offset) + ')'
sheet.range('E' + str(offset+1)).formula = '=IF(COUNTIF(E20:E' + str(offset) + ',"OPTION"), "Excluding Option", "")'
sheet.range('H' + str(offset+1)).formula = '=SUMIF(E20:E' + str(offset) + ',"<>OPTION",H20:H' + str(offset) + ')'
sheet.range('I' + str(offset+1)).formula = '=IF(H'+ str(offset+1) + '<>"", D' + str(offset+1) + '- H' + str(offset+1) + ',"")'
sheet.range('J' + str(offset+1)).formula = '=IF(I' + str(offset+1) + '<>0,I' + str(offset+1) + '/D' + str(offset+1) + ',"")'
if give_discount:
    (new_book.sheets['Config'].range('103:103')).copy(sheet.range(str(offset+2) + ':' + str(offset+2)))
    (new_book.sheets['Config'].range('104:104')).copy(sheet.range(str(offset+3) + ':' + str(offset+3)))
    sheet.range('C' + str(offset+3)).formula = '="TOTAL PROJECT PRICE AFTER DISCOUNT (" & Config!B12 & ")"'
    sheet.range('D' + str(offset+3)).formula = '=SUM(D' +str(offset+1) + ':D' + str(offset+2) + ')'
    sheet.range('H' + str(offset+3)).formula = '=$H$' +str(offset+1)
    sheet.range('I' + str(offset+3)).formula = '=IF(H'+ str(offset+3) + '<>"", D' + str(offset+3) + '- H' + str(offset+3) + ',"")'
    sheet.range('J' + str(offset+3)).formula = '=IF(I' + str(offset+3) + '<>0,I' + str(offset+3) + '/D' + str(offset+3) + ',"")'
    sheet.range('C' + str(offset+5)).value = "• All the prices are in " + quoted_currency + " excluding GST."
    sheet.range('C' + str(offset+6)).value = "• Total project price does not include prices for optional items set out in the detailed bill of material."
else:
    sheet.range('C' + str(offset+3)).formula = '="• All the prices are in " & Config!B12 & " excluding GST."'
    sheet.range('C' + str(offset+4)).value = "• Total project price does not include prices for optional items set out in the detailed bill of material."

last_row = sheet.range('C100000').end('up').row
sheet.page_setup.print_area = 'A1:F' + str(last_row+2)

In [18]:
# If the book with the same name already open, it will cuase error.
new_book.sheets['Summary'].activate()
new_book.save(downloads_folder + '/' + file_name + '.xlsx', password=hide.legacy) 
# new_book.close()